### Tuning, training and testing regularized logistic regression models for predicting college enrollment

loading packages needed

In [ ]:
# if packages not installed, install them
# import pip
# pip.main(['install', numpy]) 
# pip.main(['install', pandas]) 
# pip.main(['install', sklearn]) 
# pip.main(['install', matplotlib]) 

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
#from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier

from sklearn.externals.six import StringIO   
from sklearn.tree import export_graphviz

import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import sklearn.metrics

In [ ]:
X=np.loadtxt('dat1_collen.csv', delimiter=',') #reading in the 53 predictors for college enrolmment
print('Dimension of X is {}'.format(X.shape))
X[0:]

In [ ]:
collen=np.loadtxt('collen.csv', delimiter=',')  #reading in the college enrollment outcome variable
print('Dimension of collen is {}'.format(collen.shape))
collen[0:]

In [ ]:
weights=np.loadtxt('weights4_collen.csv', delimiter=',') #reading in public data weights at Wave IV
weights[0:20]

##### Applying nested cross-validation here (5-fold)

In [ ]:
#stratify based on the level of the outcome; then randomly split them into 5 folds for traning--testing
skf = StratifiedKFold(n_splits=5, random_state = 666, shuffle= True)
skf.get_n_splits(X, collen)
train_indices=[]
test_indices=[]
for train_index, test_index in skf.split(X, collen):
    train_indices.append(train_index)
    test_indices.append(test_index)

In [ ]:
train_indices

In [ ]:
#weighted 5-fold cross-validation
def cross_val_scores_weighted(model, X, y, weights, cv=5, metrics=[sklearn.metrics.accuracy_score]):
    skf = StratifiedKFold(n_splits=cv, random_state = 66, shuffle= True)
    skf.get_n_splits(X, y)
    scores = [[] for metric in metrics]
    for train_index, test_index in skf.split(X, y):
        model_clone = sklearn.base.clone(model)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        weights_train, weights_test = weights[train_index], weights[test_index]
        #print(weights[train_index], weights[test_index])
        model_clone.fit(X_train,y_train,sample_weight=weights_train)
        y_pred = model_clone.predict(X_test)
        for i, metric in enumerate(metrics):
            score = metric(y_test, y_pred, sample_weight = weights_test)
            scores[i].append(score)
    return scores

### train_test 1 (i.e., CV1)

In [ ]:
train1 = train_indices[0]
test1 = test_indices[0]

#### Inner-loop cross-validation: tuning regularization parameter

In [ ]:
Regs=[0.001,0.002, 0.01,0.02,0.1,0.2,1,2,10,20,100,200,1000] #1/lambda
cv1_accuracy = [] # container for training data accuracy
for Reg in Regs:
    print(Reg)
    lr = LogisticRegression(random_state = 66, C = Reg)
    scores = cross_val_scores_weighted(model=lr, X=X[train1], y=collen[train1], weights = weights[train1], cv=5)
    cv1_accuracy.append(np.mean(scores))

optimal_Reg1 = Regs[cv1_accuracy.index(max(cv1_accuracy))] 
print(optimal_Reg1)


In [ ]:
print(optimal_Reg1, max(cv1_accuracy))
plt.plot(range(len(Regs)), cv1_accuracy)
plt.ylabel('5-fold cross validation accuracy (training data)')
plt.show()

#### Back to train & test (outer loop): using the best regularization strength

In [ ]:
lr1 = LogisticRegression(random_state = 666, C = optimal_Reg1)
lr1.fit(X[train1], collen[train1], sample_weight = weights[train1])
lr1.score(X[test1], collen[test1], sample_weight=weights[test1])

In [ ]:
roc_auc_score(y_true = [ int(i) for i in collen[test1] ], 
              y_score = lr1.predict_proba(X[test1])[:, 1], sample_weight = weights[test1])
#accuracy score for CV1

### train_test 2 (i.e., CV2)

In [ ]:
train2 = train_indices[1]
test2 = test_indices[1]

####  Inner-loop cross-validation: tuning regularization parameter

In [ ]:
Regs=[0.001,0.002, 0.01,0.02,0.1,0.2,1,2,10,20,100,200,1000]
cv2_accuracy = [] # container for training data errors
for Reg in Regs:
    print(Reg)
    lr = LogisticRegression(random_state = 66, C = Reg)
    scores = cross_val_scores_weighted(model=lr, X=X[train2], y=collen[train2], weights = weights[train2], cv=5)
    cv2_accuracy.append(np.mean(scores))

optimal_Reg2 = Regs[cv2_accuracy.index(max(cv2_accuracy))] 



In [ ]:
print(optimal_Reg2, max(cv2_accuracy))
plt.plot(range(len(Regs)), cv2_accuracy)
plt.ylabel('5-fold cross validation accuracy (training data)')
plt.show()

#### Back to train & test (outer loop): using the best regularization strength

In [ ]:
lr2 = LogisticRegression(random_state = 666, C = optimal_Reg2)
lr2.fit(X[train2], collen[train2], sample_weight = weights[train2])
lr2.score(X[test2], collen[test2], sample_weight=weights[test2])

In [ ]:
roc_auc_score(y_true = [ int(i) for i in collen[test2] ], 
              y_score = lr2.predict_proba(X[test2])[:, 1], sample_weight = weights[test2])

### train_test 3 (i.e., CV3)

In [ ]:
train3 = train_indices[2]
test3 = test_indices[2]

#### Inner-loop cross-validation: tuning regularization parameter

In [ ]:
Regs=[0.001,0.002, 0.01,0.02,0.1,0.2,1,2,10,20,100,200,1000]
cv3_accuracy = [] # container for training data errors
for Reg in Regs:
    print(Reg)
    lr = LogisticRegression(random_state = 66, C = Reg)
    scores = cross_val_scores_weighted(model=lr, X=X[train3], y=collen[train3], weights = weights[train3], cv=5)
    cv3_accuracy.append(np.mean(scores))

optimal_Reg3 = Regs[cv3_accuracy.index(max(cv3_accuracy))] 
print(optimal_Reg3)


In [ ]:
print(optimal_Reg3, max(cv3_accuracy))
plt.plot(range(len(Regs)), cv3_accuracy)
plt.ylabel('5-fold cross validation accuracy (training data)')
plt.show()

#### Back to train & test (outer loop): using the best regularization strength

In [ ]:
lr3 = LogisticRegression(random_state = 666, C = optimal_Reg3)
lr3.fit(X[train3], collen[train3], sample_weight = weights[train3])
lr3.score(X[test3], collen[test3], sample_weight=weights[test3])

In [ ]:
roc_auc_score(y_true = [ int(i) for i in collen[test3] ], 
              y_score = lr3.predict_proba(X[test3])[:, 1], sample_weight = weights[test3])

### train_test 4 (i.e., CV4)

In [ ]:
train4 = train_indices[3]
test4 = test_indices[3]

#### Inner-loop cross-validation: tuning regularization parameter

In [ ]:
Regs=[0.001,0.002, 0.01,0.02,0.1,0.2,1,2,10,20,100,200,1000]
cv4_accuracy = []
for Reg in Regs:
    print(Reg)
    lr = LogisticRegression(random_state = 66, C = Reg)
    scores = cross_val_scores_weighted(model=lr, X=X[train4], y=collen[train4], weights = weights[train4], cv=5)
    cv4_accuracy.append(np.mean(scores))

optimal_Reg4 = Regs[cv4_accuracy.index(max(cv4_accuracy))] 
print(optimal_Reg4)


In [ ]:
print(optimal_Reg4, max(cv4_accuracy))
plt.plot(range(len(Regs)), cv4_accuracy)
plt.ylabel('5-fold cross validation accuracy (training data)')
plt.show()

#### Back to train & test (outer loop): using the best regularization strength

In [ ]:
lr4 = LogisticRegression(random_state = 666, C = optimal_Reg4)
lr4.fit(X[train4], collen[train4], sample_weight = weights[train4])
lr4.score(X[test4], collen[test4], sample_weight=weights[test4])

In [ ]:
roc_auc_score(y_true = [ int(i) for i in collen[test4] ], 
              y_score = lr4.predict_proba(X[test4])[:, 1], sample_weight = weights[test4])

### train_test 5 (i.e., CV5)

In [ ]:
train5 = train_indices[4]
test5 = test_indices[4]

#### Inner-loop cross-validation: tuning regularization parameter

In [ ]:
Regs=[0.001,0.002, 0.01,0.02,0.1,0.2,1,2,10,20,100,200,1000]
cv5_accuracy = [] # container for training data errors
for Reg in Regs:
    print(Reg)
    lr = LogisticRegression(random_state = 66, C = Reg)
    scores = cross_val_scores_weighted(model=lr, X=X[train5], y=collen[train5], weights = weights[train5], cv=5)
    cv5_accuracy.append(np.mean(scores))

optimal_Reg5 = Regs[cv5_accuracy.index(max(cv5_accuracy))] 
print(optimal_Reg5)


In [ ]:
print(optimal_Reg5, max(cv5_accuracy))
plt.plot(range(len(Regs)), cv5_accuracy)
plt.ylabel('5-fold cross validation accuracy (training data)')
plt.show()

#### Back to train & test (outer loop): using the best regularization strength

In [ ]:
lr5 = LogisticRegression(random_state = 666, C = optimal_Reg5)
lr5.fit(X[train5], collen[train5], sample_weight = weights[train5])
lr5.score(X[test5], collen[test5], sample_weight=weights[test5])

In [ ]:
roc_auc_score(y_true = [ int(i) for i in collen[test5] ], 
              y_score = lr5.predict_proba(X[test5])[:, 1], sample_weight = weights[test5])